In [2]:
import os
import time
import shutil
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def setup_driver(download_dir):
    options = Options()
    options.add_experimental_option("prefs", {
        "download.default_directory": os.path.abspath(download_dir),
        "download.prompt_for_download": False,
        "directory_upgrade": True,
        "safebrowsing.enabled": True
    })
    options.add_argument("--start-maximized")
    return webdriver.Chrome(options=options)

def wait_for_download_and_rename(download_dir, date):
    filename = f"pjm_lmp_{date}.csv"
    for _ in range(60):
        for file in os.listdir(download_dir):
            if file.endswith(".csv") and not file.endswith(".crdownload"):
                src = os.path.join(download_dir, file)
                dst = os.path.join(download_dir, filename)
                shutil.move(src, dst)
                return dst
        time.sleep(2)
    return None

def download_pjm_lmp_csv_selenium(date, save_dir="pjm_data"):
    os.makedirs(save_dir, exist_ok=True)
    driver = setup_driver(save_dir)
    wait = WebDriverWait(driver, 60)

    try:
        driver.get("https://dataminer2.pjm.com/feed/da_hrl_lmps")
        print(f"🌐 Opened PJM Data Miner for {date}...")

        # Wait for the input fields
        input_fields = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//input[@type='text']")))
        if len(input_fields) < 2:
            print("❌ Could not find Start and End date fields.")
            return None

        # Fill Start Date
        input_fields[0].click()
        time.sleep(1)
        for _ in range(10): input_fields[0].send_keys(Keys.BACKSPACE)
        input_fields[0].send_keys(date)

        # Fill End Date
        input_fields[1].click()
        time.sleep(1)
        for _ in range(10): input_fields[1].send_keys(Keys.BACKSPACE)
        input_fields[1].send_keys(date)

        # Click Submit
        submit_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[text()='Submit']")))
        submit_btn.click()
        print("📨 Submitted.")

        # Try clicking the export link next to "Export:"
        try:
            print("⏳ Waiting for export link...")
            export_link = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((
                By.XPATH, "//span[contains(text(), 'Export:')]/following-sibling::a"
            )))
            time.sleep(1)
            export_link.click()
            print("⬇️ Export link clicked.")
        except Exception as e:
            print("⚠️ Export link not found or failed to click.")

        wait_for_download_and_rename(save_dir, date)
        print("✅ Download complete")

    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        print("✅ Download complete")
    finally:
        driver.quit()

# Example usage
if __name__ == "__main__":
    download_pjm_lmp_csv_selenium("2024-03-01")


🌐 Opened PJM Data Miner for 2024-03-01...
📨 Submitted.
⏳ Waiting for export link...
⚠️ Export link not found or failed to click.
✅ Download complete
